## Preparing the Data

We will demonstrate fine-tuning open source models in order to classify emails into two categories, based on their content.

We will prepare 950 examples to fine-tune on, and use 50 examples to test accuracy. 



In [1]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']]
texts = [text.strip() for text in sports_dataset['data']]
df = pd.DataFrame(zip(texts, labels), columns = ['raw_prompt','response'])[:1000]
df_train = df[:950]
df_test = df[950:]

In [2]:
df_train['raw_prompt'].iloc[0]

"From: dougb@comm.mot.com (Doug Bank)\nSubject: Re: Info needed for Cleveland tickets\nReply-To: dougb@ecs.comm.mot.com\nOrganization: Motorola Land Mobile Products Sector\nDistribution: usa\nNntp-Posting-Host: 145.1.146.35\nLines: 17\n\nIn article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:\n\n|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.\n|> Does anybody know if the Tribe will be in town on those dates, and\n|> if so, who're they playing and if tickets are available?\n\nThe tribe will be in town from April 16 to the 19th.\nThere are ALWAYS tickets available! (Though they are playing Toronto,\nand many Toronto fans make the trip to Cleveland as it is easier to\nget tickets in Cleveland than in Toronto.  Either way, I seriously\ndoubt they will sell out until the end of the season.)\n\n-- \nDoug Bank                       Private Systems Division\ndougb@ecs.comm.mot.com          Motorola Communications Sect

In [4]:
print(df_train['response'].value_counts())

df_test['response'].value_counts()

response
baseball    498
hockey      452
Name: count, dtype: int64


response
baseball    25
hockey      25
Name: count, dtype: int64

In [5]:
def build_prompt(text: str):
    return f"Prompt: {text}\nCategory: "

def prepare_df(df: pd.DataFrame):
    df['prompt'] = df['raw_prompt'].apply(build_prompt)
    df.drop('raw_prompt', axis=1, inplace=True)

In [6]:
prepare_df(df_train)

/var/folders/p5/s862844923q4g7yvtkc3s9tm0000gp/T/ipykernel_91389/2775128183.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prompt'] = df['raw_prompt'].apply(build_prompt)
/var/folders/p5/s862844923q4g7yvtkc3s9tm0000gp/T/ipykernel_91389/2775128183.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('raw_prompt', axis=1, inplace=True)


In [7]:
df_train.head()

,response,prompt
0,baseball,Prompt: From: dougb@comm.mot.com (Doug Bank)\n...
1,hockey,Prompt: From: gld@cunixb.cc.columbia.edu (Gary...
2,baseball,Prompt: From: rudy@netcom.com (Rudy Wade)\nSub...
3,hockey,Prompt: From: monack@helium.gas.uug.arizona.ed...
4,baseball,Prompt: Subject: Let it be Known\nFrom: <ISSBT...


The data needs to end up in a CSV file that has two columns: `prompt` and `response`, and that is publicly accessible.

In [8]:
df_train.to_csv("sports_training_dataset.csv")

Behind the scenes, we upload our csv to `s3://scale-demo-datasets/sports/sports_training_dataset.csv`

In [23]:
# We need to set an environment variable SCALE_API_KEY. 
#   Since we're in an ipynb, we can do this. Otherwise, you just need to set the environment variable
import dotenv
env_file = ".env"
dotenv.load_dotenv(env_file, override=True)



True

In [24]:
from llmengine import FineTune, Completion, Model

FineTune.validate_api_key()

In [25]:


create_fine_tune_response = FineTune.create(
    model="llama-7b",
    training_file="s3://scale-demo-datasets/sports/sports_training_dataset.csv",
    validation_file=None,
    hyperparameters={},
    suffix="my-first-finetune"
)

fine_tune_id = create_fine_tune_response.fine_tune_id


"""
Args:
            training_file (`str`):
                A path to the file containing the training dataset.
                Dataset must be a CSV file with columns 'prompt' and 'response'.
                The value must be the URI of a publicly accessible file on bulk storage, e.g.
                s3://{public_s3_bucket}/{public_s3_key} for a file stored on s3.
            validation_file (`str`):
                A path to the file containing the validation dataset.
                Has the same format as training_file.
                If not provided, we will generate a split from the training dataset.
            model_name (`str`):
                The name of the fine-tuned model
            base_model (`str`):
                Base model to train from
            fine_tuning_method (`str`):
                Fine-tuning method
            hyperparameters (`str`):
                Hyperparameters
"""

UnauthorizedError: Invalid API Key.

In [17]:
# Wait for fine tune to complete

import time
while True:
    fine_tune_status = FineTune.retrieve(fine_tune_id).status
    if fine_tune_status == "SUCCESS":
        break
    elif fine_tune_status in ["FAILURE", "CANCELLED"]:
        raise ValueError("Fine Tune failed")
    time.sleep(10)


NameError: name 'fine_tune_id' is not defined

In [16]:
all_models = Model.list().model_endpoints

# We want to get just your fine-tuned models.
your_personal_fine_tunes = [model for model in all_models if not model.spec.public_inference]

your_fine_tuned_model = your_personal_fine_tunes[0].name

NotFoundError: Not Found

In [ ]:
response = Completion.create(model_name=your_fine_tuned_model, prompt="TODO", max_new_tokens=2)